
### 🗂️ Histórico — downloader & combinador (b3_hist_downloader.py)

**Resumo rápido**  
Script que baixa históricos de preços (via **yfinance**), salva por ticker em parquet (e opcionalmente CSV), e permite combinar todos os parquets em um único arquivo *long-format*. Feito para rodar em batch com retry/backoff, pular tickers já salvos e gerar um resumo final.


#### ✨ Principais responsabilidades
- Baixar históricos de preço por lotes (batch) com **yfinance**.  
- Salvar cada ticker em **data/historical/{TICKER}.parquet** e **{TICKER}.csv**.  
- Gerenciar retries (exponencial), fallback ticker-a-ticker e logs.  
- Gerar CSV de resumo por execução (**download_summary_YYYYMMDDTHHMMSSZ.csv**).  
- Recombinar todos os parquets em um único **all_histories.parquet** / **all_histories.csv**.


#### 🧩 Funções principais (one-liners)
- **ticker_exists_local(ticker)** → verifica existência de **{ticker}.parquet**.  
- **save_history_df(ticker, df, save_csv=True)** → salva parquet e CSV; garante coluna **date**.  
- **download_batch(batch, start, threads)** → tenta baixar um batch via **yfinance.download** com retries e fallback.  
- **download_all_histories(tickers, start, force, save_summary, save_csv_per_ticker)** → orquestra o download em batches, salva e retorna um **DataFrame** resumo.  
- **combine_all_to_single_parquet(out_path, out_csv, tickers)** → concatena todos os parquets em formato long e salva.


#### ⚙️ Parâmetros principais (valores padrão)
| Parâmetro | Valor padrão |
|---:|:---|
| **HIST_DIR** | **data/historical/** |
| **DEFAULT_START** | **"2011-01-01"** |
| **BATCH_SIZE** | **15** |
| **MAX_ATTEMPTS** | **4** |
| **SLEEP_BETWEEN_BATCHES** | **1** (seg) |
| **SLEEP_BETWEEN_TICKERS** | **0.2** (seg) |


#### 📂 Saídas geradas
- **data/historical/{TICKER}.parquet** — parquet por ticker.  
- **data/historical/{TICKER}.csv** —  CSV por ticker.  
- **data/historical/download_summary_{ts}.csv** — resumo da execução.  
- **data/historical/all_histories.parquet** & **all_histories.csv** — concat final (long format).


In [ ]:
import datetime
import logging
import os
import time

import pandas as pd

from typing import Dict, Iterable, List, Optional, Set

import yfinance as yf

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")

HIST_DIR = os.path.join("..", "data", "historical")
os.makedirs(HIST_DIR, exist_ok=True)

DEFAULT_START = "2011-01-01"
BATCH_SIZE = 15
MAX_ATTEMPTS = 4          
SLEEP_BETWEEN_BATCHES = 1  
SLEEP_BETWEEN_TICKERS = 0.2

def ticker_exists_local(ticker: str) -> bool:
    """Verifica se já existe parquet salvo para ticker (usa para pular downloads)"""
    path = os.path.join(HIST_DIR, f"{ticker}.parquet")
    return os.path.isfile(path)

def save_history_df(ticker: str, df: pd.DataFrame, save_csv: bool = True):
    """Salva DataFrame em parquet e opcionalmente em CSV. Garante coluna 'date' se índice for DatetimeIndex."""
    if df is None or df.empty:
        raise ValueError("DataFrame nulo ou vazio")
    df = df.copy()
    if isinstance(df.index, pd.DatetimeIndex):
        df.index.name = "date"
        df = df.reset_index()
    
    out_parquet = os.path.join(HIST_DIR, f"{ticker}.parquet")
    out_csv = os.path.join(HIST_DIR, f"{ticker}.csv")
    try:
        df.to_parquet(out_parquet, index=False)
        logging.info("Saved %s rows for %s -> %s", len(df), ticker, out_parquet)
    except Exception as e:
        logging.exception("Erro salvando parquet para %s: %s", ticker, e)
        raise
    if save_csv:
        try:
            if 'date' in df.columns:
                df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
            df.to_csv(out_csv, index=False)
            logging.info("Saved CSV for %s -> %s", ticker, out_csv)
        except Exception as e:
            logging.exception("Erro salvando CSV para %s: %s", ticker, e)

def download_batch(batch: List[str], start: str = DEFAULT_START, threads: bool = True) -> Dict[str, Optional[pd.DataFrame]]:
    """
    Tenta baixar um batch de tickers via yfinance.download.
    Retorna dict ticker -> DataFrame or None (se falhou).
    """
    joined = " ".join(batch)
    attempt = 0
    last_exc = None
    while attempt < MAX_ATTEMPTS:
        try:
            logging.info("yfinance.download attempt %d for batch size %d", attempt+1, len(batch))
            data = yf.download(tickers=joined, start=start, progress=False, threads=threads, group_by='ticker', auto_adjust=False, actions=True)
            result = {}
            if isinstance(data, pd.DataFrame) and isinstance(data.columns, pd.MultiIndex):
                for ticker in batch:
                    if ticker in data.columns.get_level_values(0):
                        df_t = data[ticker].copy()
                        result[ticker] = df_t
                    else:
                        try:
                            single = yf.download(ticker, start=start, progress=False, actions=True)
                            result[ticker] = single if not single.empty else None
                        except Exception:
                            result[ticker] = None
            else:
                for ticker in batch:
                    try:
                        df_t = yf.download(ticker, start=start, progress=False, actions=True)
                        result[ticker] = df_t if not df_t.empty else None
                    except Exception:
                        result[ticker] = None
            return result
        except Exception as e:
            last_exc = e
            logging.warning("Erro no yfinance.download (attempt %d): %s", attempt+1, str(e))
            attempt += 1
            time.sleep(2 ** attempt) 
    logging.error("Todas tentativas falharam para batch (%s). Último erro: %s", joined, last_exc)
    result = {}
    for ticker in batch:
        try:
            df_t = yf.download(ticker, start=start, progress=False, actions=True)
            result[ticker] = df_t if not df_t.empty else None
        except Exception as e:
            logging.warning("Fallback individual falhou para %s: %s", ticker, e)
            result[ticker] = None
    return result

def download_all_histories(tickers: List[str], start: str = DEFAULT_START, force: bool = False, save_summary: bool = True, save_csv_per_ticker: bool = True):
    """
    Processo principal: recebe lista de tickers (strings), baixa históricos e salva parquet + csv por ticker.
    - force: se True, re-baixa mesmo que arquivo exista.
    - save_csv_per_ticker: se True salva um CSV para cada ticker (além do parquet).
    - retorna um DataFrame resumo com status por ticker.
    """
    os.makedirs(HIST_DIR, exist_ok=True)
    tickers = [t for t in tickers if isinstance(t, str) and t.strip()]
    tickers = list(dict.fromkeys(tickers))
    summary = []
    for i in range(0, len(tickers), BATCH_SIZE):
        batch = tickers[i:i+BATCH_SIZE]
        to_download = [t for t in batch if force or not ticker_exists_local(t)]
        if not to_download:
            logging.info("Batch %d: todos já existem localmente — pulando.", i//BATCH_SIZE+1)
            for t in batch:
                summary.append({
                    "ticker": t,
                    "status": "skipped_local",
                    "rows": None,
                    "saved_parquet": os.path.join(HIST_DIR, f"{t}.parquet") if ticker_exists_local(t) else None,
                    "saved_csv": os.path.join(HIST_DIR, f"{t}.csv") if os.path.exists(os.path.join(HIST_DIR, f"{t}.csv")) else None
                })
            continue

        logging.info("Processando batch %d/%d (download %d/%d)", i//BATCH_SIZE+1, (len(tickers)+BATCH_SIZE-1)//BATCH_SIZE, len(to_download), len(batch))
        results = download_batch(to_download, start=start)
        for t in batch:
            df_t = results.get(t) if t in results else None
            if df_t is None or (isinstance(df_t, pd.DataFrame) and df_t.empty):
                logging.warning("Nenhum dado para %s em batch; tentativa isolada...", t)
                try:
                    single = yf.download(t, start=start, progress=False, actions=True)
                    df_t = single if not single.empty else None
                except Exception:
                    df_t = None
            if df_t is None or df_t.empty:
                logging.error("Falha obtendo dados para %s", t)
                summary.append({"ticker": t, "status": "failed", "rows": 0, "saved_parquet": None, "saved_csv": None})
            else:
                try:
                    save_history_df(t, df_t, save_csv=save_csv_per_ticker)
                    summary.append({
                        "ticker": t,
                        "status": "ok",
                        "rows": len(df_t),
                        "saved_parquet": os.path.join(HIST_DIR, f"{t}.parquet"),
                        "saved_csv": os.path.join(HIST_DIR, f"{t}.csv") if save_csv_per_ticker else None
                    })
                except Exception as e:
                    logging.exception("Erro salvando para %s: %s", t, e)
                    summary.append({"ticker": t, "status": "save_error", "rows": len(df_t) if isinstance(df_t, pd.DataFrame) else None, "saved_parquet": None, "saved_csv": None})
            time.sleep(SLEEP_BETWEEN_TICKERS)
        time.sleep(SLEEP_BETWEEN_BATCHES)

    df_summary = pd.DataFrame(summary)
    if save_summary:
        ts = datetime.datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
        summary_path = os.path.join(HIST_DIR, f"download_summary_{ts}.csv")
        df_summary.to_csv(summary_path, index=False)
        logging.info("Resumo salvo em %s", summary_path)
    return df_summary

def combine_all_to_single_parquet(out_path: str = os.path.join(HIST_DIR, "all_histories.parquet"), out_csv: Optional[str] = os.path.join(HIST_DIR, "all_histories.csv"), tickers: Optional[List[str]] = None):
    """
    Lê todos os parquets em HIST_DIR (ou tickers list) e concatena em formato long:
    columns: ['ticker','date', 'Open','High','Low','Close','Adj Close','Volume', 'Dividends','Stock Splits']
    Salva em parquet e opcionalmente em csv.
    """
    files = []
    if tickers:
        files = [os.path.join(HIST_DIR, f"{t}.parquet") for t in tickers if os.path.exists(os.path.join(HIST_DIR, f"{t}.parquet"))]
    else:
        files = [os.path.join(HIST_DIR, f) for f in os.listdir(HIST_DIR) if f.endswith(".parquet")]
    dfs = []
    for f in files:
        try:
            df = pd.read_parquet(f)
            if 'date' in df.columns:
                df['date'] = pd.to_datetime(df['date'])
            fname = os.path.basename(f).replace(".parquet","")
            if 'ticker' not in df.columns:
                df.insert(0, 'ticker', fname)
            dfs.append(df)
        except Exception as e:
            logging.warning("Erro lendo %s: %s", f, e)
    if not dfs:
        raise RuntimeError("Nenhum parquet encontrado para combinar.")
    big = pd.concat(dfs, ignore_index=True, sort=False)
    big.to_parquet(out_path, index=False)
    logging.info("Combined saved to %s (rows=%d)", out_path, len(big))
    if out_csv:
        try:
            if 'date' in big.columns:
                big['date'] = pd.to_datetime(big['date']).dt.strftime('%Y-%m-%d')
            big.to_csv(out_csv, index=False)
            logging.info("Combined CSV saved to %s", out_csv)
        except Exception as e:
            logging.exception("Erro salvando combined CSV: %s", e)
    return big

if __name__ == "__main__":
    tickers_file = os.path.join("..", "data", "tickers_ibrx100_full.csv")
    if os.path.exists(tickers_file):
        df = pd.read_csv(tickers_file)
        if 'Ticker' in df.columns:
            tickers = df['Ticker'].dropna().astype(str).tolist()
        else:
            tickers = df.iloc[:,0].dropna().astype(str).tolist()
    else:
        raise RuntimeError(f"Não encontrou {tickers_file}. Coloque seu CSV de tickers na pasta 'aurum/data/' ou edite este script.")

    def normalize(t):
        t = str(t).strip().upper()
        if not t.endswith(".SA"):
            t = t.replace(".SA","") + ".SA"
        return t
    tickers = [normalize(t) for t in tickers]
    print("Tickers a baixar:", len(tickers), tickers[:10])

    summary_df = download_all_histories(tickers, start=DEFAULT_START, force=False, save_summary=True, save_csv_per_ticker=True)
    print(summary_df.head(50))

    combined = combine_all_to_single_parquet()
    print("Combined rows:", len(combined))

2025-12-11 17:58:44,565 INFO Processando batch 1/7 (download 15/15)
2025-12-11 17:58:44,573 INFO yfinance.download attempt 1 for batch size 15


Tickers a baixar: 97 ['ALOS3.SA', 'ABEV3.SA', 'ANIM3.SA', 'ASAI3.SA', 'AURE3.SA', 'AXIA3.SA', 'AXIA6.SA', 'AZZA3.SA', 'B3SA3.SA', 'BBSE3.SA']


2025-12-11 17:58:48,183 INFO Saved 3716 rows for ALOS3.SA -> ..\data\historical\ALOS3.SA.parquet
2025-12-11 17:58:48,244 INFO Saved CSV for ALOS3.SA -> ..\data\historical\ALOS3.SA.csv
2025-12-11 17:58:48,480 INFO Saved 3716 rows for ABEV3.SA -> ..\data\historical\ABEV3.SA.parquet
2025-12-11 17:58:48,585 INFO Saved CSV for ABEV3.SA -> ..\data\historical\ABEV3.SA.csv
2025-12-11 17:58:48,811 INFO Saved 3716 rows for ANIM3.SA -> ..\data\historical\ANIM3.SA.parquet
2025-12-11 17:58:48,885 INFO Saved CSV for ANIM3.SA -> ..\data\historical\ANIM3.SA.csv
2025-12-11 17:58:49,105 INFO Saved 3716 rows for ASAI3.SA -> ..\data\historical\ASAI3.SA.parquet
2025-12-11 17:58:49,180 INFO Saved CSV for ASAI3.SA -> ..\data\historical\ASAI3.SA.csv
2025-12-11 17:58:49,399 INFO Saved 3716 rows for AURE3.SA -> ..\data\historical\AURE3.SA.parquet
2025-12-11 17:58:49,453 INFO Saved CSV for AURE3.SA -> ..\data\historical\AURE3.SA.csv
2025-12-11 17:58:49,668 INFO Saved 3716 rows for AXIA3.SA -> ..\data\historical\

       ticker status  rows                         saved_parquet  \
0    ALOS3.SA     ok  3716   ..\data\historical\ALOS3.SA.parquet   
1    ABEV3.SA     ok  3716   ..\data\historical\ABEV3.SA.parquet   
2    ANIM3.SA     ok  3716   ..\data\historical\ANIM3.SA.parquet   
3    ASAI3.SA     ok  3716   ..\data\historical\ASAI3.SA.parquet   
4    AURE3.SA     ok  3716   ..\data\historical\AURE3.SA.parquet   
5    AXIA3.SA     ok  3716   ..\data\historical\AXIA3.SA.parquet   
6    AXIA6.SA     ok  3716   ..\data\historical\AXIA6.SA.parquet   
7    AZZA3.SA     ok  3716   ..\data\historical\AZZA3.SA.parquet   
8    B3SA3.SA     ok  3716   ..\data\historical\B3SA3.SA.parquet   
9    BBSE3.SA     ok  3716   ..\data\historical\BBSE3.SA.parquet   
10   BBDC3.SA     ok  3716   ..\data\historical\BBDC3.SA.parquet   
11   BBDC4.SA     ok  3716   ..\data\historical\BBDC4.SA.parquet   
12   BRAP4.SA     ok  3716   ..\data\historical\BRAP4.SA.parquet   
13   BBAS3.SA     ok  3716   ..\data\historical\

2025-12-11 17:59:38,825 INFO Combined saved to ..\data\historical\all_histories.parquet (rows=360452)
2025-12-11 17:59:42,805 INFO Combined CSV saved to ..\data\historical\all_histories.csv


Combined rows: 360452


#### 📋 Descrição das colunas do dataset histórico

| Coluna         | Tipo        | Breve descrição |
|---------------:|:-----------:|:----------------|
| **date**         | **date**      | Data da observação no formato ISO (**YYYY-MM-DD**). Use como índice temporal para séries. |
| **Open**         | **float**     | Preço de abertura do pregão (primeira transação considerada naquele dia). |
| **High**         | **float**     | Preço máximo registrado durante o pregão. |
| **Low**          | **float**     | Preço mínimo registrado durante o pregão. |
| **Close**        | **float**     | Preço de fechamento (última transação do dia). Não considera ajustes por eventos corporativos. |
| **Adj Close**    | **float**     | Preço de fechamento **ajustado** por splits e dividendos — use para cálculo de retornos total/consistentes em séries históricas. |
| **Volume**       | **int**       | Quantidade de ações negociadas no dia (unidades). |
| **Dividends**    | **float**     | Valor do dividendo pago por ação na data (se houver). Geralmente em moeda local (ex.: BRL para B3). |
| **Stock Splits** | **float**     | Fator de desdobramento/agrupamento (ex.: **2.0** → split 2-por-1). Zero ou **0.0** quando não houve evento. |

**Notas rápidas**
- **Adj Close** é a coluna recomendada para backtests e cálculo de retornos contínuos (corrige preços para manter consistência após eventos corporativos).  
- Converta **date** para **datetime** e defina como índice para operações de série temporal (**df['date'] = pd.to_datetime(df['date']); df.set_index('date', inplace=True)**).  
- Verifique a unidade/moeda conforme a fonte (normalmente moeda local do mercado, ex.: BRL para B3).  



---

### 📊 Documentação: Ferramenta de Análise Exploratória de Histórico

#### 📝 Visão Geral

Este script (**analyze_dataframe**) atua como uma **ferramenta de diagnóstico** dentro do pipeline de dados do **Projeto Aurum**.

Seu objetivo principal é validar a integridade do dataset mestre de preços (**all_histories.csv**) logo após o processo de unificação dos dados históricos. Ele serve para garantir que o arquivo CSV consolidado foi gerado corretamente antes que ele seja utilizado em etapas críticas como o cálculo de indicadores ou backtesting.



### ⚙️ Funcionalidades Principais

O script executa uma auditoria sequencial no arquivo de dados:

1.  **Verificação de Existência:** Checa se o arquivo existe no caminho especificado antes de tentar carregar, evitando erros em tempo de execução.
2.  **Carregamento Otimizado:** Lê o CSV convertendo automaticamente a coluna de datas para o formato **datetime** (essencial para séries temporais).
3.  **Auditoria de Estrutura:**
      * Mostra tipos de dados (**dtypes**).
      * Identifica valores nulos (**NaN**) por coluna.
      * Exibe estatísticas descritivas (Média, Mínimo, Máximo, Desvio Padrão) para preços e volumes.
4.  **Validação de Tickers:** Conta e lista os ativos únicos presentes no arquivo para garantir que a consolidação abrangeu todo o universo (ex: IBRX-100).


### 🔍 Detalhamento do Código

#### Bibliotecas Utilizadas

  * **pandas**: Motor principal para manipulação e análise tabular.
  * **os**: Utilizado para verificar caminhos e existência de arquivos no sistema operacional.
  * **io**: Utilizado especificamente (**io.StringIO**) para capturar o output da função **df.info()**, que nativamente imprime no console, permitindo manipulá-lo como string se necessário.

### Funções

#### 1\. **print_header(title)**

Uma função auxiliar estética.

  * **Objetivo:** Criar separadores visuais no log do console.
  * **Utilidade:** Facilita a leitura rápida do relatório quando executado em terminais com muito texto.

#### 2\. **analyze_dataframe(file_path)**

A função *core* do script. Executa os seguintes passos:

  * **Tratamento de Erros:** Envolve o carregamento (**pd.read_csv**) em um bloco **try-except** para capturar arquivos corrompidos ou mal formatados.
  * **Parsing de Datas:** Usa o argumento **parse_dates=['date']**. Isso é crucial para o Aurum, pois permite operações de data (ex: filtrar últimos 5 anos) sem precisar converter strings manualmente depois.
  * **Detecção de Nulos:**
    **python
    null_counts = df.isnull().sum()
    if null_counts.sum() == 0: ...
    **
    Isso é vital para dados financeiros. **Nulls** em preços de fechamento quebram backtests. Este bloco avisa imediatamente se há buracos nos dados.
  * **Verificação de Tickers:** Confirma quantos ativos únicos existem. Se você espera 100 ações (IBRX-100) e o script retorna 50, você sabe que houve erro na etapa de coleta.


#### 🚀 Como Utilizar

1.  Certifique-se de que o arquivo consolidado existe no diretório:
    **../data/historical/all_histories.csv**
2.  Execute o script via terminal:
    ```bash
    python analise_historico.py
    ```
3.  **Interpretação do Output:**
      * **Sucesso:** O script imprimirá as 5 primeiras linhas, o resumo de memória e confirmará "Não há valores nulos".
      * **Atenção:** Se houver valores nulos em `Close` ou `Adj Close`, você deve revisitar o script de limpeza de dados.


### 🔗 Contexto no Projeto Aurum

Dentro da arquitetura do Aurum, este script se encaixa na etapa de **Validação de Qualidade de Dados (Data Quality)**.

```mermaid
flowchart LR
    A[Coleta Yahoo Finance] --> B[Dataset Bruto]
    B --> C[Limpeza e Unificação]
    C --> D[(all_histories.csv)]
    D --> E{Script de Análise}
    E -- OK --> F[Cálculo de Indicadores]
    E -- Erro --> G[Revisar Coleta]
```

-----

*Documentação gerada automaticamente para o Laboratório Quantitativo Aurum.*

In [ ]:
import pandas as pd
import os
import io

def print_header(title):
    """
    Função auxiliar para imprimir um cabeçalho formatado no log.
    """
    print("\n" + "=" * 70)
    print(f" {title.upper()} ")
    print("=" * 70)

def analyze_dataframe(file_path):
    """
    Carrega e analisa um DataFrame de histórico de ações.
    """
    
    print(f"--- Iniciando Análise do Arquivo: {file_path} ---")
    
    if not os.path.exists(file_path):
        print_header("[ERRO] Arquivo não encontrado")
        print(f"O arquivo no caminho '{file_path}' não foi localizado.")
        print("Por favor, verifique se o caminho está correto e a pasta 'historical' existe.")
        print("=" * 70)
        return

    try:
        df = pd.read_csv(file_path, parse_dates=['date'])
        print(f"\n[SUCESSO] Arquivo carregado. Total de {len(df)} linhas e {len(df.columns)} colunas.")
    except Exception as e:
        print_header("[ERRO] Falha ao carregar o arquivo")
        print(f"Ocorreu um erro ao tentar ler o arquivo CSV: {e}")
        print("=" * 70)
        return

    print_header("1. Amostra dos Dados (Primeiras 5 Linhas)")
    print(df.head().to_string())

    print_header("2. Informações do DataFrame (Tipos de Coluna e Nulos)")
    buffer = io.StringIO()
    df.info(buf=buffer)
    info_str = buffer.getvalue()
    print(info_str)

    print_header("3. Resumo de Valores Nulos por Coluna")
    null_counts = df.isnull().sum()
    
    if null_counts.sum() == 0:
        print("Ótimo! Não há valores nulos em nenhuma coluna.")
    else:
        print(null_counts[null_counts > 0].to_string())

    print_header("4. Estatísticas Descritivas (Colunas Numéricas)")
    try:
        print(df.describe().to_string())
    except Exception as e:
        print(f"Não foi possível calcular estatísticas descritivas: {e}")

    print_header("5. Análise da Coluna 'ticker'")
    if 'ticker' in df.columns:
        unique_tickers = df['ticker'].unique()
        num_unique_tickers = len(unique_tickers)
        print(f"Total de tickers únicos encontrados: {num_unique_tickers}")
        
        if num_unique_tickers > 10:
            print(f"Amostra de tickers: {unique_tickers[:10]}...")
        else:
            print(f"Tickers presentes: {unique_tickers}")
    else:
        print("Coluna 'ticker' não encontrada.")

    print("\n" + "=" * 70)
    print("--- ANÁLISE CONCLUÍDA ---")
    print("=" * 70)

if __name__ == "__main__":
    path_do_arquivo = '../data/historical/all_histories.csv'
    analyze_dataframe(path_do_arquivo)

--- Iniciando Análise do Arquivo: ../data/historical/all_histories.csv ---

[SUCESSO] Arquivo carregado. Total de 360452 linhas e 10 colunas.

 1. AMOSTRA DOS DADOS (PRIMEIRAS 5 LINHAS) 
     ticker       date      Open      High       Low     Close  Adj Close     Volume  Dividends  Stock Splits
0  ABEV3.SA 2011-01-03  8.632311  8.728203  8.630313  8.690246   4.694567   576145.0        0.0           0.0
1  ABEV3.SA 2011-01-04  8.784141  8.784141  8.630313  8.692244   4.695647   328368.0        0.0           0.0
2  ABEV3.SA 2011-01-05  8.672266  8.718215  8.448517  8.530425   4.608231   299836.0        0.0           0.0
3  ABEV3.SA 2011-01-06  8.560392  8.590358  8.396576  8.450515   4.565063   731319.0        0.0           0.0
4  ABEV3.SA 2011-01-07  8.450515  8.550403  8.368607  8.416553   4.546715  1090222.0        0.0           0.0

 2. INFORMAÇÕES DO DATAFRAME (TIPOS DE COLUNA E NULOS) 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360452 entries, 0 to 360451
Data columns (tota


-----

### 🧹 ETL: Sanitização de Dados Históricos de Mercado

#### 📝 Visão Geral

Este script é responsável pela etapa de **Limpeza e Normalização (Data Cleaning)** dos dados brutos de preços (OHLCV) coletados.

Ele atua como um filtro de qualidade, removendo registros inconsistentes, dias sem negociação ou dados corrompidos antes que eles entrem no pipeline de cálculo de indicadores ou backtesting. Isso é crucial para evitar erros de divisão por zero ou distorções estatísticas.

-----

### ⚙️ Fluxo de Execução

O script segue uma lógica de **Fallback** (tentativa e erro) para o carregamento e uma lógica estrita para a limpeza.

```mermaid
flowchart LR
    Start([Início]) --> TryParquet{Existe<br/>.parquet?}
    
    TryParquet -- Sim --> LoadP[Carregar Parquet]
    TryParquet -- Não --> TryCSV{Existe<br/>.csv?}
    
    TryCSV -- Sim --> LoadC[Carregar CSV]
    TryCSV -- Não --> Error[❌ Erro Fatal]
    
    LoadP --> Clean[Sanitização]
    LoadC --> Clean
    
    subgraph Sanitização [Regras de Limpeza]
        R1[Drop NaN em 'Adj Close']
        R2[Drop NaN em 'Volume']
        R3[Filtro: Volume > 0]
    end
    
    Clean --> Save[Salvar Arquivos Limpos]
    Save --> End([Fim])
```

-----

### 🔍 Detalhes da Implementação

#### 1\. Ingestão Inteligente (Input)

O script utiliza um mecanismo robusto de carregamento:

  * **Prioridade:** Tenta carregar o formato `.parquet` (mais rápido e eficiente em memória).
  * **Fallback:** Se o Parquet não existir, recorre automaticamente ao `.csv` bruto.
  * **Parser:** Ao carregar o CSV, já converte a coluna de data (`parse_dates=['date']`), garantindo a tipagem correta.

#### 2\. Regras de Limpeza (Business Logic)

O script aplica dois filtros rigorosos para garantir a integridade do dataset `df_prices_clean`:

1.  **Remoção de Nulos (`dropna`):**
      * Remove linhas onde o Preço Ajustado (`Adj Close`) ou Volume sejam nulos/NaN.
      * *Motivo:* Dados nulos quebram cálculos de médias móveis e indicadores técnicos.
2.  **Filtro de Liquidez (`Volume > 0`):**
      * Remove dias onde o volume negociado foi zero (feriados locais onde a bolsa mundial abriu, dias de suspensão de negociação do ativo, ou erros de coleta).
      * *Motivo:* Dias com volume zero distorcem a média de liquidez e podem gerar sinais falsos de estabilidade de preço.

#### 3\. Persistência (Output)

Os dados limpos são salvos no diretório `../data/historical/` em dois formatos simultaneamente:

  * `all_histories_cleaned.parquet`: Para leitura rápida nos próximos scripts do Python.
  * `all_histories_cleaned.csv`: Para inspeção visual rápida (Excel/Notepad) ou depuração.

-----

#### 📊 Métricas de Qualidade

O script gera logs informativos (`logging`) que permitem monitorar a "saúde" dos dados:

  * **Contagem de Linhas:** Informa quantas linhas existiam antes e quantas restaram.
  * **Perda de Dados:** Calcula e exibe quantas linhas foram descartadas (`linhas_removidas`). Uma taxa de remoção muito alta pode indicar problemas na fonte de dados (Yahoo Finance/B3).

-----

### 🚀 Como Executar

Certifique-se de que o arquivo bruto (`all_histories.parquet` ou `.csv`) exista na pasta de dados.

```bash
python clean_market_data.py
```

### Exemplo de Saída no Console:

```text
2025-12-13 16:30:00 - INFO - Dados de preço brutos carregados do PARQUET: 150000 linhas
2025-12-13 16:30:01 - INFO - Dados de preço limpos: 148500 linhas (removidas 1500 linhas com NaN ou Volume 0)
2025-12-13 16:30:02 - INFO - ✅ Arquivo limpo salvo em (Parquet): ../data/historical/all_histories_cleaned.parquet
2025-12-13 16:30:02 - INFO - ✅ Arquivo limpo salvo em (CSV): ../data/historical/all_histories_cleaned.csv

Limpeza concluída. 1500 linhas removidas.
```

-----

*Documentação do módulo ETL do Projeto Aurum.*

In [ ]:
import pandas as pd
import numpy as np
import logging
import os

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

try:
    df_prices_raw = pd.read_parquet("../data/historical/all_histories.parquet")
    logger.info(f"Dados de preço brutos carregados do PARQUET: {df_prices_raw.shape[0]} linhas")
except FileNotFoundError:
    logger.warning("Arquivo all_histories.parquet não encontrado! Tentando carregar o CSV...")
    try:
        df_prices_raw = pd.read_csv("../data/historical/all_histories.csv", parse_dates=['date'])
        logger.info(f"Dados de preço brutos carregados do CSV: {df_prices_raw.shape[0]} linhas")
    except FileNotFoundError as e:
        logger.error("Nenhum arquivo de histórico (parquet ou csv) encontrado. Execute o download primeiro.")
        raise e
except Exception as e:
    logger.error(f"Erro ao carregar dados: {e}")
    raise

df_prices_clean = df_prices_raw.dropna(subset=['Adj Close', 'Volume'])

df_prices_clean = df_prices_clean[df_prices_clean['Volume'] > 0]

linhas_removidas = len(df_prices_raw) - len(df_prices_clean)
logger.info(f"Dados de preço limpos: {df_prices_clean.shape[0]} linhas (removidas {linhas_removidas} linhas com NaN ou Volume 0)")

output_dir = "../data/historical"
os.makedirs(output_dir, exist_ok=True)

parquet_path = os.path.join(output_dir, "all_histories_cleaned.parquet")
csv_path = os.path.join(output_dir, "all_histories_cleaned.csv")

try:
    df_prices_clean.to_parquet(parquet_path, index=False)
    logger.info(f"✅ Arquivo limpo salvo em (Parquet): {parquet_path}")

    df_prices_clean.to_csv(csv_path, index=False, date_format='%Y-%m-%d')
    logger.info(f"✅ Arquivo limpo salvo em (CSV): {csv_path}")

except Exception as e:
    logger.error(f"❌ Erro ao salvar arquivos limpos: {e}")

print(f"\nLimpeza concluída. {linhas_removidas} linhas de 'lookahead' removidas.")
print(df_prices_clean.head())

2025-12-13 15:56:26,391 - INFO - Dados de preço brutos carregados do PARQUET: 360452 linhas
2025-12-13 15:56:26,456 - INFO - Dados de preço limpos: 276324 linhas (removidas 84128 linhas com NaN ou Volume 0)
2025-12-13 15:56:26,738 - INFO - ✅ Arquivo limpo salvo em (Parquet): ../data/historical\all_histories_cleaned.parquet
2025-12-13 15:56:30,192 - INFO - ✅ Arquivo limpo salvo em (CSV): ../data/historical\all_histories_cleaned.csv



Limpeza concluída. 84128 linhas de 'lookahead' removidas.
     ticker       date      Open      High       Low     Close  Adj Close  \
0  ABEV3.SA 2011-01-03  8.632311  8.728203  8.630313  8.690246   4.694567   
1  ABEV3.SA 2011-01-04  8.784141  8.784141  8.630313  8.692244   4.695647   
2  ABEV3.SA 2011-01-05  8.672266  8.718215  8.448517  8.530425   4.608231   
3  ABEV3.SA 2011-01-06  8.560392  8.590358  8.396576  8.450515   4.565063   
4  ABEV3.SA 2011-01-07  8.450515  8.550403  8.368607  8.416553   4.546715   

      Volume  Dividends  Stock Splits  
0   576145.0        0.0           0.0  
1   328368.0        0.0           0.0  
2   299836.0        0.0           0.0  
3   731319.0        0.0           0.0  
4  1090222.0        0.0           0.0  



-----

### 🔄 ETL: Transformação e Pivoteamento de Preços (Mensal)

#### 📝 Visão Geral

Este script é responsável por **alterar a dimensionalidade** dos dados de mercado, preparando o terreno para a etapa de Backtesting e geração de sinais.

Ele transforma o dataset "Longo" (onde cada linha é um registro diário de um ticker) em datasets "Wide" (Matrizes), onde o índice é a Data, as colunas são os Tickers e os valores são os preços. Além disso, ele realiza a **reamostragem temporal** (Resampling) de dados diários para **mensais**.

-----

### ⚙️ Fluxo de Transformação

O script realiza uma conversão crucial de formato de dados:

**De: Formato Longo (Transactional)**
| Data | Ticker | Adj Close |
| :--- | :--- | :--- |
| 2024-01-01 | PETR4 | 35.00 |
| 2024-01-01 | VALE3 | 70.00 |
| 2024-01-02 | PETR4 | 36.00 |

**Para: Formato Wide (Matrix)**
| Data (Index) | PETR4 | VALE3 | ... |
| :--- | :--- | :--- | :--- |
| 2024-01-31 | 38.00 | 72.00 | ... |
| 2024-02-29 | 39.50 | 71.00 | ... |

```mermaid
flowchart LR
    Input[("<b>all_histories_cleaned.parquet</b><br/>(Dados Diários Longos)")] --> Process
    
    subgraph Process [Processamento]
        A["<b>Resample Mensal ('M')</b><br/>Captura o último preço do mês"]
        B["<b>Resample Mensal ('MS')</b><br/>Captura o preço de abertura do mês"]
        C["<b>Pivot Table</b><br/>Transforma Tickers em Colunas"]
        D["<b>Fill NA</b><br/>Preenchimento de lacunas (ffill/bfill)"]
    end
    
    Input --> A & B
    A --> C
    B --> C
    C --> D
    
    D --> Out1[("<b>prices_close_wide.parquet</b><br/>Matriz de Fechamentos")]
    D --> Out2[("<b>prices_open_wide.parquet</b><br/>Matriz de Aberturas")]
```

-----

### 🔍 Detalhes da Lógica

#### 1\. Reamostragem (Resampling)

Para simular uma estratégia de rebalanceamento mensal, não precisamos de 252 dias úteis por ano, apenas do preço de entrada e saída de cada mês.

  * **Fechamento (`Close`):** Usa a frequência `'M'` (Month End) e a função `.last()`. Pega o preço do último dia de negociação do mês.
  * **Abertura (`Open`):** Usa a frequência `'MS'` (Month Start) e a função `.first()`. Pega o preço do primeiro dia de negociação do mês.

#### 2\. Tratamento de Lacunas (Gap Filling)

Matrizes de preços não podem ter buracos (NaN) para cálculos vetoriais. O script aplica uma estratégia agressiva de preenchimento:

  * **Forward Fill (`ffill`):** Se um ativo não foi negociado em um mês específico, ele repete o preço do mês anterior.
  * **Backward Fill (`bfill`):** Se o ativo não tinha dados no início do histórico, ele puxa o primeiro preço válido para trás (para evitar NaNs no começo da série).

-----

### 📂 Entradas e Saídas

#### Input

  * **Arquivo:** `../data/historical/all_histories_cleaned.parquet`
  * **Conteúdo:** Histórico diário limpo de todos os ativos (formato longo).

#### Outputs

Salvos em `../data/historical/`:

1.  **`prices_close_wide.parquet`**:
      * Matriz contendo apenas os preços de fechamento ajustados (`Adj Close`) de fim de mês.
      * Essencial para calcular a rentabilidade da carteira.
2.  **`prices_open_wide.parquet`**:
      * Matriz contendo os preços de abertura (`Open`) de início de mês.
      * Utilizado para simular o preço de execução da compra no rebalanceamento.

-----

### 🚀 Como Executar

Este script depende da execução prévia do script de limpeza.

```bash
python gerar_matriz_precos.py
```

### Exemplo de Log de Sucesso:

```text
2025-12-13 16:45:00 - INFO - Iniciando o Passo 1: Geração dos Preços Pivotados (Wide)...
2025-12-13 16:45:01 - INFO - Dados limpos carregados.
2025-12-13 16:45:02 - INFO - Reamostrando para Frequência Mensal (M e MS)...
2025-12-13 16:45:03 - INFO - Corrigindo MultiIndex e Pivotando...
2025-12-13 16:45:04 - INFO - ✅ ARQUIVO FALTOSO GERADO: ../data/historical/prices_close_wide.parquet
```

-----

*Documentação do módulo de Transformação de Dados do Projeto Aurum.*

In [1]:
import pandas as pd
import numpy as np
import logging
import os

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

INPUT_FILE = "../data/historical/all_histories_cleaned.parquet"
OUTPUT_DIR = "../data/historical"

def gerar_precos_pivotados():
    logger.info("Iniciando o Passo 1: Geração dos Preços Pivotados (Wide)...")
    
    try:
        df_prices_clean = pd.read_parquet(INPUT_FILE)
        logger.info(f"Dados limpos '{INPUT_FILE}' carregados.")
    except FileNotFoundError:
        logger.error(f"ARQUIVO NÃO ENCONTRADO: {INPUT_FILE}")
        logger.error("Execute o 'Script 1: Limpeza' primeiro.")
        return

    df_prices_clean['date'] = pd.to_datetime(df_prices_clean['date'])

    logger.info("Reamostrando para Frequência Mensal (M e MS)...")
    df_prices_mensal_raw = df_prices_clean.set_index('date').groupby('ticker').resample('M').last()

    df_open_mensal_raw = df_prices_clean.set_index('date').groupby('ticker').resample('MS').first()

    logger.info("Corrigindo MultiIndex e Pivotando...")
    
    df_prices_mensal_long = df_prices_mensal_raw.drop(columns='ticker', errors='ignore').reset_index()
    df_open_mensal_long = df_open_mensal_raw.drop(columns='ticker', errors='ignore').reset_index()

    df_close_wide = df_prices_mensal_long.pivot(index='date', columns='ticker', values='Adj Close')
    df_open_wide = df_open_mensal_long.pivot(index='date', columns='ticker', values='Open')

    logger.info("Sincronizando e preenchendo índices de data...")
    idx_union = df_close_wide.index.union(df_open_wide.index)
    
    df_close_wide = df_close_wide.reindex(idx_union, method='ffill')
    df_open_wide = df_open_wide.reindex(idx_union, method='ffill')

    df_close_wide = df_close_wide.ffill().bfill()
    df_open_wide = df_open_wide.ffill().bfill()

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    path_close = os.path.join(OUTPUT_DIR, "prices_close_wide.parquet")
    path_open = os.path.join(OUTPUT_DIR, "prices_open_wide.parquet")
    
    df_close_wide.to_parquet(path_close)
    df_open_wide.to_parquet(path_open)
    
    logger.info(f"✅ ARQUIVO FALTOSO GERADO: {path_close}")
    logger.info(f"✅ ARQUIVO FALTOSO GERADO: {path_open}")
    print("\n--- Amostra de Fechamento (Wide) ---")
    display(df_close_wide.head())

if __name__ == "__main__":
    gerar_precos_pivotados()



2025-12-23 15:35:00,506 - INFO - Iniciando o Passo 1: Geração dos Preços Pivotados (Wide)...
2025-12-23 15:35:00,909 - INFO - Dados limpos '../data/historical/all_histories_cleaned.parquet' carregados.
2025-12-23 15:35:00,927 - INFO - Reamostrando para Frequência Mensal (M e MS)...
C:\Users\kaike\AppData\Local\Temp\ipykernel_492\1996424201.py:26: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_prices_mensal_raw = df_prices_clean.set_index('date').groupby('ticker').resample('M').last()
C:\Users\kaike\AppData\Local\Temp\ipykernel_492\1996424201.py:26: FutureWarning: DataFrameGroupBy.resample operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_prices_mensal_raw = df_prices_clean.set_index(


--- Amostra de Fechamento (Wide) ---


ticker,ABEV3.SA,ALOS3.SA,ANIM3.SA,ASAI3.SA,AURE3.SA,AXIA3.SA,AXIA6.SA,AZZA3.SA,B3SA3.SA,BBAS3.SA,...,TOTS3.SA,UGPA3.SA,USIM5.SA,VALE3.SA,VAMO3.SA,VBBR3.SA,VIVA3.SA,VIVT3.SA,WEGE3.SA,YDUQ3.SA
date,,,,,,,,,,,,,,,,,,,,,
2011-01-01,4.097764,32.057922,5.146394,14.210447,12.438302,12.054853,10.141385,14.825628,6.694447,5.331043,...,8.352773,2.950102,14.937788,23.735777,7.560618,12.779866,22.094751,6.297232,2.167035,5.368343
2011-01-31,4.097764,32.057922,5.146394,14.210447,12.438302,12.054853,10.141385,14.825628,6.694447,5.331043,...,8.352773,2.950102,14.937788,23.735777,7.560618,12.779866,22.094751,6.297232,2.167035,5.368343
2011-02-01,4.097764,32.057922,5.146394,14.210447,12.438302,12.054853,10.141385,14.825628,6.694447,5.331043,...,8.352773,2.950102,14.937788,23.735777,7.560618,12.779866,22.094751,6.297232,2.167035,5.368343
2011-02-28,4.041646,32.057922,5.146394,14.210447,12.438302,12.646833,11.386950,14.825628,6.503195,5.386558,...,8.092393,2.950102,14.607204,23.411423,7.560618,12.779866,22.094751,6.543143,2.205022,5.435447
2011-03-01,4.041646,32.057922,5.146394,14.210447,12.438302,12.646833,11.386950,14.825628,6.503195,5.386558,...,8.092393,2.950102,14.607204,23.411423,7.560618,12.779866,22.094751,6.543143,2.205022,5.435447
